In [ ]:
from langsmith import traceable
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
if api_key is None:
    raise ValueError("Please set the GOOGLE_GENAI_API_KEY environment variable.")

os.environ["LANGSMITH_TRACING_V2"] = os.getenv("LANGSMITH_TRACING", "true")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")

llm = init_chat_model(
    "gemini-2.0-flash",
    model_provider="google_genai",
    api_key=api_key
  )


In [9]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
"""
Kamu adalah konselor profesional yang penuh empati. Berdasarkan konteks berikut, berikan respons singkat, manusiawi, dan berfokus pada inti masalah pengguna.
Gunakan bahasa yang sederhana, suportif, dan tidak menggurui.

Informasi tambahan: 
{context}

Pertanyaan dari pengguna:
{question}

Setelah memberikan sedikit pemahaman atau dukungan, akhiri dengan kalimat yang membuka ruang untuk pengguna berbagi lebih banyak atau memperdalam pembicaraan.
"""
)


In [11]:
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
import torch

# Load SBERT model dan pastikan menggunakan GPU jika tersedia
device = 'cuda' if torch.cuda.is_available() else 'cpu'
sbert_model = model = SentenceTransformer('naufalihsan/indonesian-sbert-large')
sbert_model = sbert_model.to(device)  # Pindahkan model ke GPU (jika ada)

# Custom embeddings class for SBERT
class SBERTEmbeddings(Embeddings):
    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        # Menggunakan model SBERT untuk menghasilkan embeddings
        embeddings = sbert_model.encode(texts, convert_to_tensor=True, show_progress_bar=True)
        embeddings = embeddings.to(device)  # Pindahkan embeddings ke GPU (jika ada)
        return embeddings.cpu().numpy().tolist()  # Pindahkan kembali ke CPU untuk konversi

    def embed_query(self, query: str) -> list[float]:
        # Menghasilkan embedding untuk query
        embedding = sbert_model.encode(query, convert_to_tensor=True)
        embedding = embedding.to(device)  # Pindahkan embedding ke GPU (jika ada)
        return embedding.cpu().numpy().tolist()  # Pindahkan kembali ke CPU untuk konversi

# Inisialisasi embeddings SBERT dan FAISS vector store
sbert_embeddings = SBERTEmbeddings()

d:\CODING\PYTHON\MACHINE_LEARNING\Tenangin\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\CODING\PYTHON\MACHINE_LEARNING\Tenangin\.venv\Lib\site-packages\transformers\utils\hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [12]:
def serialize_context(context_list):
    serialized = ""
    for idx, doc in enumerate(context_list, start=1):
        serialized += f"[Dokumen {idx}]\nJudul: {doc['BAB']}\nIsi: {doc['isi']}\n\n"
    return serialized


In [22]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langsmith import traceable


vector_store = FAISS.load_local("Embeddings",sbert_embeddings,allow_dangerous_deserialization=True)


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"],k=2)
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = [
    {
        "BAB": doc.metadata.get("bab", "Tidak diketahui"),
        "isi": doc.page_content
    }
    for doc in state["context"]
   ]
    serialized_context = serialize_context(docs_content)
    messages = prompt.invoke({"question": state["question"], "context": serialized_context})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [23]:
response = graph.invoke({"question": "saya merasa kesepian, bagaimana cara mengatasinya?"})
print(f"Anda    : {response['question']}")
print(f"Gemini  : {response['answer']}")
print(f"Context : {response['context']}")

Anda    : saya merasa kesepian, bagaimana cara mengatasinya?
Gemini  : Kesepian itu berat ya. Dari yang kamu ceritakan, sepertinya kamu sedang merasa terisolasi dan butuh koneksi. Penting untuk diingat bahwa perasaan ini wajar dan banyak orang mengalaminya.

Mencari cara untuk terhubung kembali dengan orang lain bisa membantu. Mungkin dengan menghubungi teman lama, mengikuti kegiatan yang kamu sukai, atau mencari komunitas yang memiliki minat yang sama. Ingat, langkah kecil pun tetap langkah maju.

Apakah ada hal spesifik yang membuatmu merasa kesepian akhir-akhir ini? Atau adakah hal yang sudah kamu coba lakukan untuk mengatasinya? Ceritakan lebih banyak, ya. Aku di sini untuk mendengarkan.
Context : [Document(id='4c03f4c2-009f-4e0e-9236-09a5e52769c8', metadata={'bab': 'BAB VIII MOOD DAN BUNUH DIRI'}, page_content='Kebanyakan orang yang melakukan bunuh diri memiliki gangguan mood, dan mereka yang menderita gangguan mood memiliki risiko kematian akibat bunuh diri yang lebih tinggi dari